In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
# Clean up the URLs
def excel_to_df(filepath):
    # Import data from excel sheet
    with warnings.catch_warnings(record=True):
        warnings.simplefilter("always")
        df = pd.read_excel(filepath, sheet_name='Dataset1', engine="openpyxl")

    # Clean up column names
    df.rename(columns={
        'Page path level 2':'url',
        'Pageviews': 'pageviews',
        'Unique Pageviews': 'unique_pageviews',
        'Avg. Time on Page': 'avg_time',
        'Bounce Rate': 'bounce_rate',
        '% Exit': 'exit%'
        }, inplace=True)

    # Clean up blog URLs to match formatting of previous data
    df['url'] = 'blog' + df['url'].str.split('?', expand=True)[0].str.rstrip('/')
    return df

In [25]:
# Import all three excel sheets
df1 = excel_to_df('../data/raw/Google_analytics_1.xlsx')
print(f'size of df1: {df1.size}')
df2 = excel_to_df('../data/raw/Google_analytics_2.xlsx')
print(f'size of df2: {df2.size}')
df3 = excel_to_df('../data/raw/Google_analytics_3.xlsx')
print(f'size of df3: {df3.size}')

# Stitch the dataframes together
df = pd.concat([df1, df2, df3], axis=0).reset_index()
print(f'size of combined dataframe: {df.size}')

# Aggregate data along blog URLs
wm = lambda x: np.average(x, weights=df.loc[x.index, "pageviews"])
df = df.groupby('url').agg(
    {
        'pageviews': 'sum',
        'unique_pageviews': 'sum',
        'avg_time': wm,
        'bounce_rate' : wm,
        'exit%': wm
    }
)

#Write into a CSV file
df.to_csv('../data/google_analytics.csv')

size of df1: 30006
size of df2: 30006
size of df3: 18036
size of combined dataframe: 91056
